In [0]:
#liên kết Google Colab và Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from itertools import chain
import pickle
import time

class main:
    def __init__(self):
        
        """
        num_users: số lượng user
        num_items: số lượng item
        k: số lượng factor
        mu: muy
        threshold: giá trị ngưỡng để xét sự yêu thích của user với item
        """
        self.groups=[]
        self.k =15
        self.lamda = 0.055 #regulariztion
        self.alpha = 0.05 # learning_rate
        self.mu = 0
        
        self.train_data=[]
        self.test_data=[]
        self.ratings=[]
        self.test_ratings=[]

        self.num_users = 0
        self.num_items = 0
        
        self.threshold = 4
        
        """
        Chạy offline
        """
        #self.normalize_data()
        """
        Chạy online
        """
        url_ratings =  "/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/ratings.txt"
        url_test_rating =  "/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/test_ratings.txt"
        self.test_ratings = np.loadtxt(url_test_rating)
        self.test_ratings = np.array(self.test_ratings)
        self.ratings = np.loadtxt(url_ratings)

        self.num_users = len(self.ratings)
        self.num_items = len(self.ratings[0])
           
        
        """
        Khởi tạo user_bias, item_bias bằng 0
        """
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.mu = np.mean(self.ratings[np.where(self.ratings != 0)]) #rating trung bình toàn cục
        
        """
        Khởi tạo ma trận user factor-P, item factor-Q theo phân phối chuẩn 
        """
        self.P = np.random.normal(scale=1./self.k, size=(self.num_users, self.k))
        self.Q = np.random.normal(scale=1./self.k, size=(self.num_items, self.k))

        
        """
        Khởi tạo ma trận dự đoán: ma trận full sau khi thực hiện sgd
        """
        #self.predictions = np.zeros((self.num_users,self.num_items))
        
        
        """
        Danh sách rating của người dùng ảo
        """
        self.virtual_rating = []
        
        """
        Danh sách lưu lại các giá trị dự đoán đánh giá cho group
        """
        self.pred_for_group_AF = []
        self.pred_for_group_BF = []
        self.pred_for_group_WBF = []
        
        """
        Danh sách lưu lại các item dùng để recommend cho group theo phương pháp tương ứng
        """
        self.recommend_list_AF = []
        self.recommend_list_BF = []
        self.recommend_list_WBF = []
        
        """
        Tạo mảng precision, recall theo group size
        """
        self.eva_small = []
        self.eva_med = []
        self.eva_large = []

In [0]:
"""
Đọc data và đưa dữ liệu vào ma trận rating
"""

def normalize_data(self):
    
    dataset_url = "/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/ratings.csv"
    dataset = pd.read_csv(dataset_url)
    
    # chia tập train 70% , test 30%
    train_data, test_data = train_test_split(dataset,test_size=0.3,random_state=42)
    
    #xác định số lượng user và item
    self.num_users = max(train_data.userId.unique())
    self.num_items = max(train_data.movieId.unique())
    
    
    #khởi tạo ma trận training và test ban đầu bằng ma trận 0
    self.ratings = np.zeros((self.num_users, self.num_items), dtype=np.int8)
    self.test_ratings = np.zeros((self.num_users, self.num_items), dtype=np.int8)

    #đọc dữ liệu từ tập training vào ma trận rating ban đầu
    for row in train_data.itertuples(index = False):
        self.ratings[row.userId - 1, row.movieId - 1] = row.rating
        
    #đọc dữ liệu từ tập test vào ma trận test
    for row in test_data.itertuples(index = False):
        self.test_ratings[row.userId - 1, row.movieId - 1] = row.rating
main.normalize_data = normalize_data

In [0]:
"""
#Ví dụ giải thích 2 giá trị ratings_row, ratings_col trong hàm sgd
import numpy as np
a = np.array([[1, 0, 1],
              [2, 3, 4],
              [0, 0, 0]])

ratings_row, ratings_col = a.nonzero() # lấy mảng index của rating khác 0
print(ratings_row)
print(ratings_col)
"""


'\n#Ví dụ giải thích 2 giá trị ratings_row, ratings_col trong hàm sgd\nimport numpy as np\na = np.array([[1, 0, 1],\n              [2, 3, 4],\n              [0, 0, 0]])\n\nratings_row, ratings_col = a.nonzero() # lấy mảng index của rating khác 0\nprint(ratings_row)\nprint(ratings_col)\n'

In [0]:
def sgd(self):
    ratings_row, ratings_col = self.ratings.nonzero() # lấy mảng index của rating khác 0
    num_ratings = np.count_nonzero(self.ratings) # tổng số rating khác 0
    rating_indices = np.arange(num_ratings)
    max_iter = 500
    for iter in range(max_iter):   
        for idx in rating_indices:
            user = ratings_row[idx]
            item = ratings_col[idx]
        
            """
            Tính error theo Công thức số 28 trong paper
            """
        
            error = self.ratings[user][item] - self.predict_for_user(user, item) 
    
            """
            Cập nhật user factor, item factor, user bias, item bias theo Công thức 29 -> 32 trong paper
            """    
            
            # Sao chép giá trị cũ của self.P[user] để tính self.Q[item] vì self.Q[item] được tính dựa
            # trên giá trị cũ của self.P[user]
            P_u = self.P[user]
            
            self.P[user] += self.alpha * ((error * self.Q[item]) - (self.lamda * self.P[user])) 
            self.Q[item] += self.alpha * ((error * P_u) - (self.lamda * self.Q[item]))
            self.b_u[user] += self.alpha * (error - self.lamda * self.b_u[user])
            self.b_i[item] += self.alpha * (error - self.lamda * self.b_i[item])  
           
        #self.mse()
main.sgd = sgd

In [0]:
"""
Tính giá trị rating dự đoán cho 1 user theo công thức số [2] trong paper
"""
def predict_for_user(self, user, item):
    return self.mu + self.b_u[user] + self.b_i[item] + self.P[user, :].dot(self.Q[item, :].T)
main.predict_for_user = predict_for_user    

In [0]:
"""
Điền full ma trận dự đoán sau bước sgd
"""
def predicted_matrix(self):
    for user in range(self.num_users):
        for item in range(self.num_items):
            self.predictions[user, item] = self.predict_for_user(user, item)
main.predicted_matrix = predicted_matrix

In [0]:
"""
Tính rating dự đoán của group cho 1 item theo công thức số [3] trong paper
"""
def predict_for_group_for_item(self, group, item):
    #3 tính mG
    return self.mu + group.group_bias + self.b_i[item] + np.dot(group.group_factor.T, self.Q[item])
main.predict_for_group_for_item = predict_for_group_for_item

In [0]:
def mse(self):
    b.predicted_matrix()
    return mean_squared_error(self.predictions[self.ratings.nonzero()], self.ratings[self.ratings.nonzero()])
main.mse = mse


## Tạo nhóm

In [0]:
"""
Class chứa các thuộc tính của 1 group
    users : danh sách id các member trong group
    size: số lượng member trong group
"""
class group:
    def __init__(self, users, size):
        self.users = users
        self.size = size
        self.group_factor = []
        self.group_bias = 0

In [0]:
"""
Hàm đếm số item được tất cả user rate dùng cho việc generate group
"""
def manual_rating(self, group):
    group_rating = self.ratings[group, :]
    count = 0
    for item in range(self.num_items):
        if len(group) == np.count_nonzero(group_rating[:,item]):
            count+=1
    return count
main.manual_rating = manual_rating

In [0]:
def generate_group(self, size):
    loop = self.num_users
    temp_group = []
    temp_manual_rating = [] 
    num_group = 0
    while loop >= size:
        # random group theo size mà không trùng user giữa các group
        g = np.random.choice(range(self.num_users), size = size, replace = False)
        # tính số lượng rating chung giữa các user trong group để ranking và lấy top k groups
        temp_manual_rating.append(self.manual_rating(g))
        # mảng chứa tất cả các group được tạo
        temp_group.append(g)
        loop -= size
        num_group += 1
    
    # ranking group từ thấp đến cao theo số lượng rating chung giữa các user
    # [-int(num_group/60):]: top k group cần lấy, dấu - : lấy từ dưới lên trên
    index = sorted(range(len(temp_manual_rating)), key=lambda x: temp_manual_rating[x])[-30:]
    
    #chuyển temp_group từ list thành mảng
    temp_group_array = np.array(temp_group)
    #lấy ra top k group
    top_group = temp_group_array[index]
    #thêm vào mảng self.groups
    for gp in top_group:
        self.groups+=[group(gp,size)]
main.generate_group = generate_group    

In [0]:
"""
Hàm đánh giá, tính recall, precision
"""
def evaluation(self, group, method): 
    tp = 0
    fp = 0
    t = 0

    if method == 'AF':
        #pred = self.pred_for_group_AF[-1] #lấy group cuối cùng trong mảng self.pred_for_group_AF
        rG = self.recommend_list_AF[-1]
    elif method == 'BF':
        #pred = self.pred_for_group_BF[-1]
        rG = self.recommend_list_BF[-1]
    else:
        #pred = self.pred_for_group_WBF[-1]
        rG = self.recommend_list_WBF[-1]
    
    # tính false positive: số lượng những item
    temp_fp=[]
    #print('rG: ',rG)
    logical_fp = np.argwhere(np.logical_and(self.test_ratings[group.users[0],:] < self.threshold,\
                                         self.test_ratings[group.users[0],:] > 0)).flatten()
    temp_fp = np.intersect1d(logical_fp,rG)
   
    for user in group.users[1:]:
      logical_fp = np.argwhere(np.logical_and(self.test_ratings[user,:] < self.threshold,\
                                         self.test_ratings[user,:] > 0)).flatten()
      temp_fp_ = np.intersect1d(logical_fp,rG)
      temp_fp=np.union1d(temp_fp,temp_fp_)    
  
    fp= temp_fp.size

    
    # tính true positive
    temp_tp=[]
    logical_tp= np.argwhere(np.logical_or(self.test_ratings[group.users[0],:] >= self.threshold,\
                                       self.test_ratings[group.users[0],:] == 0)).flatten() # tập item thỏa < ngưỡng của user[0] trong group
    
    temp_tp = np.intersect1d(logical_tp,rG)
    for user in group.users[1:]: #chạy từ user thứ 2 trong group
      logical_tp= np.argwhere(np.logical_or(self.test_ratings[user,:] >= self.threshold,\
                                       self.test_ratings[user,:] == 0)).flatten()
      temp_tp_ = np.intersect1d(logical_tp,rG)
      temp_tp=np.intersect1d(temp_tp,temp_tp_)
  
                             #giao lại để lấy tập item
  
    tp = temp_tp.size

    #tính expected recommendations sets: tập những đề xuất được mong đợi: chứa các item >= ngưỡng dù
    #chưa được đánh giá và hay đánh giá rồi
    
    temp_t= np.argwhere(np.logical_or(self.test_ratings[group.users[0],:] >= self.threshold,\
                                       self.test_ratings[group.users[0],:] == 0)).flatten() # tập item thỏa < ngưỡng của user[0] trong group
    for user in group.users[1:]: #chạy từ user thứ 2 trong group
      temp_t = np.intersect1d(temp_t,np.argwhere(np.logical_or(self.test_ratings[user,:] >= self.threshold,\
                                                                 self.test_ratings[user,:] == 0)).flatten())
                               #giao lại để lấy tập item  
    unrate_test_rating = np.nonzero(np.sum(self.test_ratings[[group.users]],axis = 0))
    non_test_rating = np.setdiff1d(range(self.num_items), unrate_test_rating)
    temp_t = np.setdiff1d(temp_t, non_test_rating)   
    t=temp_t.size
   
    try:
        precision_value = tp / (tp + fp)
    except ZeroDivisionError:
        precision_value = np.NaN
        
    try:
        recall_value = tp/t
    except ZeroDivisionError:
        recall_value = np.NaN    
    #print('pre: ',precision_value)
    #print('recall: ',recall_value)
    #lưu lại các giá trị precision, recall của từng group theo group size
    if group.size < 5:
        self.eva_small.append([precision_value, recall_value, method])
    elif group.size < 9:
        self.eva_med.append([precision_value, recall_value, method])
    else:
        self.eva_large.append([precision_value, recall_value, method])
        
main.evaluation = evaluation

In [0]:
  """
  import numpy as np
  values = np.array([
    [8,0,3,1,5,0],
    [3,6,6,7,2,0],
    [3,4,5,0,1,0],
    [6,4,0,7,8,0]])
  
  print(values[[1,2,3],1] <5)
  """

'\nimport numpy as np\nvalues = np.array([\n  [8,0,3,1,5,0],\n  [3,6,6,7,2,0],\n  [3,4,5,0,1,0],\n  [6,4,0,7,8,0]])\n\nprint(values[[1,2,3],1] <5)\n'

In [0]:
"""
Tính ma trận A: ma trận chứa các item factor của các item được users trong group rated
"""
def matrix_A(self, group, method_agg):
    """
    Tính virtual rating cho group theo method_agg tương ứng
    
    """
    self.cal_virtual_rating(group, method_agg)

    """
    Khởi tạo ma trận A - ma trận item factors của group 
    """
    A = self.Q[[j for j, e in enumerate(self.virtual_rating) if e != 0], :]
    temp = np.ones((np.count_nonzero(self.virtual_rating),1))
    A = np.hstack((A,temp)) # thêm 1 cột vào ma trận item factor của group
    return A
main.matrix_A  = matrix_A

In [0]:
"""
Tính sG theo định nghĩa của công thức số [11] trong paper
"""
def cal_sG(self):
    sG = []
    # chỉ tính sG cho những item đã được virtual user rate
    for item in [j for j, e in enumerate(self.virtual_rating) if e != 0]:
      value = self.virtual_rating[item] - self.mu - self.b_i[item]
      
      sG.append(value)
    return sG
main.cal_sG = cal_sG

In [0]:
"""
Tính rating dự đoán của group cho tất của item
""" 
def predict_for_group(self, group, method):
    tmp_pred_group = []
    if method == 'BF':
        for item in range(self.num_items):
            metric = self.predict_for_group_for_item(group,item)
            tmp_pred_group.append(metric)
        self.pred_for_group_BF.append(tmp_pred_group)
    elif method == 'WBF':
        for item in range(self.num_items):
            metric = self.predict_for_group_for_item(group,item)
            tmp_pred_group.append(metric)
        self.pred_for_group_WBF.append(tmp_pred_group)
    else:
        for item in range(self.num_items):
            metric = self.predict_for_group_for_item(group,item)
            tmp_pred_group.append(metric)
        self.pred_for_group_AF.append(tmp_pred_group)
main.predict_for_group = predict_for_group

In [0]:
"""
Hàm tính toán virtual rating dựa theo paper: tính virtual user dựa trên ma trận ratings ban đầu
"""
def cal_virtual_rating(self, group, method_agg):
        """
        Tính group rating hay rating của virtual user
        group_rating: ma trận rating của group
        virtual_rating: rating chung của cả group theo method_agg[most_pleasure, least_misery, average_weight]
        """
        self.virtual_rating = []
        group_rating = self.ratings[group.users, :]
        #print('group_rating:\n',group_rating)
        #most_pleasure lấy giá trị lớn nhất
        if method_agg == 'most_pleasure':
            self.virtual_rating = np.max(group_rating,axis=0)
        # least_misery lấy giá trị nhỏ nhất khác 0
        
        elif method_agg == 'least_misery':
            self.virtual_rating = np.min(group_rating,axis=0)
            for i in range(self.num_items):
                if self.virtual_rating[i] ==0:
                  rated_item = np.unique(group_rating[:,i])
                  if len(rated_item) != 1:
                    self.virtual_rating[i] =  rated_item[1]
                  else:
                    self.virtual_rating[i] =  0 # lấy các giá trị unique trong mảng, r lấy
                                                # giá trị nhỏ thứ 2 trong mảng (để tránh lấy số 0)
                    
        # tính trung bình cộng có weight  
        else:
            num_ratings_users = (group_rating != 0).sum(1) #mảng chứa số lượng rating khác 0 của từng user trong group
            sum_num_ratings_users = sum(num_ratings_users)
            for item in range(self.num_items):
                sum_rating_weight = 0 # 1x2+3x4(1,3:rating của user;2,4 là số lượng rating của user tương ứng trên all item) 
                for i, user in enumerate(group.users):
                    sum_rating_weight += self.ratings[user][item] * num_ratings_users[i]
                
                average_weight = sum_rating_weight / sum_num_ratings_users
                self.virtual_rating.append(average_weight)
            
        """
            
        else: 
            self.virtual_rating = np.true_divide(group_rating.sum(0),(group_rating!=0).sum(0)) #chia theo cột không tính số 0 - tính 
                                                                                    #trung bình rating g
            self.virtual_rating = np.nan_to_num(self.virtual_rating)
        """

main.cal_virtual_rating =cal_virtual_rating    

In [0]:
"""
Hàm tính toán virtual rating dựa theo phương pháp chạy MF,tính virtual user dựa trên ma trận predictions(full rating)
"""
def cal_virtual_rating(self, group, method_agg):
        """
        Tính group rating hay rating của virtual user
        group_rating: ma trận rating của group
        virtual_rating: rating chung của cả group theo method_agg[most_pleasure, least_misery, average_weight]
        """
        self.virtual_rating = []
        group_rating = self.ratings[group.users, :]
        #most_pleasure lấy giá trị lớn nhất
        if method_agg == 'most_pleasure':
            self.virtual_rating = np.max(group_rating,axis=0)
        # least_misery lấy giá trị nhỏ nhất khác 0
        
        elif method_agg == 'least_misery':
            self.virtual_rating = np.min(group_rating,axis=0)
            for i in range(self.num_items):
                if self.virtual_rating[i] ==0:
                  rated_item = np.unique(group_rating[:,i])
                  if len(rated_item) != 1:
                    self.virtual_rating[i] =  rated_item[1]
                  else:
                    self.virtual_rating[i] =  0 # lấy các giá trị unique trong mảng, r lấy
                                                # giá trị nhỏ thứ 2 trong mảng (để tránh lấy số 0)
                    
        # tính trung bình cộng có weight  
        else:
            num_ratings_users = (group_rating != 0).sum(1) #mảng chứa số lượng rating khác 0 của từng user trong group
            sum_num_ratings_users = sum(num_ratings_users)
            for item in range(self.num_items):
                sum_rating_weight = 0 # 1x2+3x4(1,3:rating của user;2,4 là số lượng rating của user tương ứng trên all item) 
                for i, user in enumerate(group.users):
                    sum_rating_weight += self.ratings[user][item] * num_ratings_users[i]
                
                average_weight = sum_rating_weight / sum_num_ratings_users
                self.virtual_rating.append(average_weight) 
        """
            
        else: 
            self.virtual_rating = np.true_divide(group_rating.sum(0),(group_rating!=0).sum(0)) #chia theo cột không tính số 0 - tính 
                                                                                    #trung bình rating g
            self.virtual_rating = np.nan_to_num(self.virtual_rating)
        """

main.cal_virtual_rating =cal_virtual_rating    

In [0]:
"""
#Hàm tính virtual user theo phương pháp chỉ tính những item mà tất cả user của group đều rate
nếu k thì là unobserved rating
"""

def cal_virtual_rating(self, group, method_agg):
        """
        Tính group rating hay rating của virtual user
        group_rating: ma trận rating của group
        virtual_rating: rating chung của cả group theo method_agg[most_pleasure, least_misery, average_weight]
        """
        self.virtual_rating = []
        group_rating = self.ratings[group.users, :]
        # nếu item không được tất cả user trong group rate thì gán rating mà tất cả users trong group rate cho item đó =0
        for item in range(self.num_items):
            if group.size != np.count_nonzero(group_rating[:,item]):
                for user in range(group.size):
                    group_rating[user, item] = 0
        self.virtual_rating = []        
        if method_agg == 'most_pleasure':
            self.virtual_rating = np.max(group_rating,axis=0)
        elif method_agg == 'least_misery':
            self.virtual_rating = np.min(group_rating,axis=0)
            for i in range(self.num_items):
                if self.virtual_rating[i] ==0:
                  rated_item = np.unique(group_rating[:,i])
                  if len(rated_item) != 1:
                    self.virtual_rating[i] =  rated_item[1]
                  else:
                    self.virtual_rating[i] =  0 # lấy các giá trị unique trong mảng, r lấy
                                                # giá trị nhỏ thứ 2 trong mảng (để tránh lấy số 0)
        # tính trung bình cộng có weight                
        else:
            num_ratings_users = (group_rating != 0).sum(1) #mảng chứa số lượng rating khác 0 của từng user trong group
            sum_num_ratings_users = sum(num_ratings_users)
            #print(num_ratings_users)
            for item in range(self.num_items):
                sum_rating_weight = 0 # 1x2+3x4(1,3:rating của user;2,4 là số lượng rating của user tương ứng trên all item) 
                for i in range(group.size):
                    sum_rating_weight += group_rating[i][item] * num_ratings_users[i]

                
                average_weight = sum_rating_weight / sum_num_ratings_users
                self.virtual_rating.append(average_weight)
            
        """
            
        else: 
            self.virtual_rating = np.true_divide(group_rating.sum(0),(group_rating!=0).sum(0)) #chia theo cột không tính số 0 - tính 
                                                                                    #trung bình rating g
            self.virtual_rating = np.nan_to_num(self.virtual_rating)
        """
                    
main.cal_virtual_rating =cal_virtual_rating    

In [0]:
"""
Tạo danh sách item chưa được đánh giá bởi bất kỳ người dùng nào trong group
"""
def unrate_item(self, group):
    #unrate_tuple = np.where(self.virtual_rating == 0) #tính unrate cho group dựa trên virtual_rating
    #unrate_2darr = np.asarray(unrate_tuple)
    #unrate = unrate_2darr.ravel()
    rated_train = np.nonzero(np.sum(self.ratings[[group.users]],axis = 0))
    unrate = np.setdiff1d(range(self.num_items), rated_train)  
    return unrate
main.unrate_item = unrate_item

In [0]:

"""
Tạo danh sách đề xuất cho group theo method AF, BF, WBF dựa theo điều kiện [4,5,6] trong paper
"""
def recommend_list(self, group, pred, method,unrate):
  

    
    """
    Xác định các item đã được rate: rated_item = num_items - unrate_item
    """
    
    unrate_test_rating = np.nonzero(np.sum(self.test_ratings[[group.users]],axis = 0))
    non_test_rating = np.setdiff1d(range(self.num_items), unrate_test_rating)
    actual_unrate = np.setdiff1d(unrate, non_test_rating)
    
    """
    Lấy giá trị rating lớn nhất trong tập đã được rate 
    """
    
    group_rating = np.array(pred[-1])    
    temp_recommend_list=[]
    sorted_gr_rating = np.argsort(group_rating) #ranking từ thấp tới cao vẫn giữ nguyên index
    for item in sorted_gr_rating:
      if item in actual_unrate:
        temp_recommend_list.append(item)
    #print('re_list',temp_recommend_list)
    #print('unrate: ',unrate)
    print(group_rating[[2990, 1658, 7022, 45081, 2467, 1266, 5902, 1921, 3082, 2617]])
    temp_recommend_list = temp_recommend_list[-10:][::-1]
    
    """
    So sánh giá trị rating của các item unrate trong group prediction rồi gán vào danh sách tương ứng
    """
    #temp_recommend_list = []
    #for item in unrate:
    #  print('so sanh', group_rating[item], 'va' , max_rated_item_rating)
    #  if group_rating[item] >= max_rated_item_rating: #[6]
    #    temp_recommend_list.append(item)

    if method == 'AF':
        self.recommend_list_AF.append(temp_recommend_list)
    elif method == 'BF':
        self.recommend_list_BF.append(temp_recommend_list)
    elif method == 'WBF':
        self.recommend_list_WBF.append(temp_recommend_list)    
        
main.recommend_list = recommend_list

In [0]:
def AF(self, method_agg):
    self.eva_small = []
    self.eva_med = []
    self.eva_large = []
    for group in self.groups:
      
        users_factor = self.P[group.users, :]
        users_bias = self.b_u[group.users]
        group_rating = self.ratings[group.users, :]
        
        if method_agg == 'most_pleasure':
            group.group_factor = np.max(users_factor,axis=0)
            group.group_bias = np.max(users_bias,axis=0)
        elif method_agg == 'least_misery':
            group.group_factor = np.min(users_factor,axis=0)
            group.group_bias = np.min(users_bias,axis=0)
        else:
            num_ratings_users = (group_rating != 0).sum(1) #mảng chứa số lượng rating khác 0 của từng user trong group
            sum_num_ratings_users = sum(num_ratings_users)
            group_factor = []
            for factor in range(self.k):
                sum_factor_weight = 0 
                sum_bias_weight = 0
                for i, user in enumerate(group.users):
                    sum_factor_weight += self.P[user][factor] * num_ratings_users[i]
                    sum_bias_weight += self.b_u[user] * num_ratings_users[i]

                group_factor_value = sum_factor_weight / sum_num_ratings_users
                group_bias = sum_bias_weight / sum_num_ratings_users
                group_factor.append(group_factor_value)
                group.group_bias = group_bias
            group.group_factor = np.array(group_factor)
               
        
        """
        Cập nhật ma trận dự đoán
        """
        self.predict_for_group(group, 'AF')
        
        """
        Tạo danh sách item chưa được đánh giá bởi bất kỳ người dùng nào trong group
        """
        self.cal_virtual_rating(group, 'most_pleasure') # chỉ chạy cell tính virtual rating theo paper
        unrate = self.unrate_item(group)
        
        """
        Tạo danh sách đề xuất
        """
        self.recommend_list(group, self.pred_for_group_AF, 'AF',unrate)
        
        #print('group_factor:\n',group.group_factor)
        #print('group_bias:\n',group.group_bias)
        
        #print('ma tran du doan group:\n', self.pred_for_group_AF)
        #print('recommend list:\n',self.recommend_list_AF)
        
        """
        Đánh giá
        """
        self.evaluation(group, 'AF')
        
        print('Done!')
    
    #ghi xuống file các giá trị đánh giá
    np.savetxt('eva_small_' + method_agg + '.txt', self.eva_small,fmt='%s')
    np.savetxt('eva_med_' + method_agg + '.txt', self.eva_med,fmt='%s')
    np.savetxt('eva_large_'+ method_agg + '.txt', self.eva_large,fmt='%s')
        
main.AF = AF

In [0]:
def BF(self, method_agg):
    self.eva_small = []
    self.eva_med = []
    self.eva_large = []
    for group in self.groups:
        
        """
        Tính ma trận A
        """
        A = self.matrix_A(group, method_agg) #trong A đã gọi hàm cal_virtual_user
        
        #print(A)

        """
        Tính sG
        """
        sG = self.cal_sG()
        
        """
        Tính pG - group factor, bG - bias group, Thêm vào user factor, user bias 
        """
        #pG_bG = np.dot(np.linalg.inv(np.dot(np.dot(A.T, A) + lamb * np.identity(self.k + 1)),np.dot(A.T, sG))
        #factor_n_bias = np.dot(np.linalg.inv(np.dot(A.T, A) + lamb * np.identity(self.cfg.num_factors + 1)), np.dot(A.T, s_g))
        
        pG_bG = np.dot(np.linalg.inv(np.dot(A.T,A) + self.lamda * np.identity(self.k + 1)),np.dot(A.T, sG))
        group.group_factor = pG_bG[:-1] # lấy group factor từ pG_bG bỏ dòng cuối
        group.group_bias = pG_bG[-1] # lấy dòng cuối từ pG_bG

        """
        Tính rating dự đoán của group cho tất cả item
        """
        self.predict_for_group(group, 'BF')
        unrate = self.unrate_item(group)
            
        """
        Tạo danh sách đề xuất
        """
        self.recommend_list(group, self.pred_for_group_BF, 'BF',unrate)
        
        #print('virtual_user:\n',self.virtual_rating)
        #print('ma tran A:\n',A)
        #print('sG:\n',sG)
        #print('group_factor:\n',group.group_factor)
        #print('group_bias:\n', group.group_bias)
        #print('ma tran du doan group:\n', self.pred_for_group_BF)
        #print('recommend list:\n',self.recommend_list_BF)
        
        
        """
        Đánh giá
        """
        self.evaluation(group, 'BF')
        
    #ghi xuống file các giá trị đánh giá 
    #with open('eva_small_' + method_agg + '.txt', 'w') as small:
     # pickle.dump(self.eva_small, small,pickle.HIGHEST_PROTOCOL)
    #with open('eva_med_' + method_agg + '.txt', 'w') as med:
     # pickle.dump(self.eva_med, med,pickle.HIGHEST_PROTOCOL)
    #with open('eva_large_' + method_agg + '.txt', 'w') as large:
     # pickle.dump(self.eva_large, large,pickle.HIGHEST_PROTOCOL)
    
    np.savetxt('eva_small_' + method_agg + '.txt', self.eva_small,fmt='%s')
    np.savetxt('eva_med_' + method_agg + '.txt', self.eva_med,fmt='%s')
    np.savetxt('eva_large_'+ method_agg + '.txt', self.eva_large,fmt='%s')
main.BF = BF

In [0]:
def WBF(self, method_agg):
    self.eva_small = []
    self.eva_med = []
    self.eva_large = []
    #w_value_arr = []
    """
    online
    """
    #w_value_arr_url = "w_value_arr.csv"
    #w_value_arr = pd.read_csv(w_value_arr_url)
    #w_value_arr = w_value_arr.to_numpy()
    for index, group in enumerate(self.groups):
        group_rating = self.ratings[group.users, :]
        """
        Tính ma trận A
        """
        A = self.matrix_A(group, method_agg)
        

        """
        Tính sG
        """
        sG = self.cal_sG()
        
        """
        Tạo ma trận trọng số W theo công thức số [15] trong paper
        """
        """offline"""
        rated_item = np.nonzero(np.sum(group_rating,axis = 0))
        W_value = []
        for item in rated_item[0]:
          standard_deviation = np.std(group_rating[:,item]) #tính độ lệch chuẩn của tất cả member's rating trong 
                                                                # group cho item i
          num_rated = (group_rating != 0).sum(0)[item] / group.size
          W_value.append(num_rated * (1 / (1 + standard_deviation)))
        W = np.diag(W_value)  #chuyển thành ma trận đường chéo, có đường chéo chính là W_value
        
        """
        online 
        """
        #W_value = w_value_arr[index][~np.isnan(w_value_arr[index])][1:]
        #W = np.diag(W_value)
        
        """
        Tính pG - group factor, bG - bias group, Thêm vào user factor, user bias 
        """
        pG_bG = np.dot(np.linalg.inv(np.dot(np.dot(A.T, W),A) + self.lamda * np.identity(self.k + 1)),np.dot(np.dot(A.T, W), sG))
        group.group_factor = pG_bG[:-1] # lấy group factor từ #14 bỏ dòng cuối
        group.group_bias = pG_bG[-1]
        
        """
        Tính rating dự đoán của group cho tất cả item
        """
        self.predict_for_group(group, 'WBF')
        unrate = self.unrate_item(group)
            
        """
        Tạo danh sách đề xuất
        """
        self.recommend_list(group, self.pred_for_group_WBF, 'WBF',unrate)
        
        #print('virtual_user:\n',self.virtual_rating)
        #print('ma tran A:\n',A)
        #print('sG:\n',sG)
        #print('W:\n',W)
        #print('group_factor:\n',group.group_factor)
        #print('group_bias:\n', group.group_bias)
        #print('ma tran du doan group:\n', self.pred_for_group_WBF)
        #print('recommend list:\n',self.recommend_list_WBF)
        
        """
        Đánh giá
        """
        self.evaluation(group, 'WBF')
        print('Done!') 
        
    """offline"""
    #pd.concat([pd.DataFrame(w_value_arr)], axis=1).to_csv('w_value_arr.csv')
    #ghi xuống file các giá trị đánh giá
    np.savetxt('eva_small_' + method_agg + '.txt', self.eva_small,fmt='%s')
    np.savetxt('eva_med_' + method_agg + '.txt', self.eva_med,fmt='%s')
    np.savetxt('eva_large_'+ method_agg + '.txt', self.eva_large,fmt='%s')
   
main.WBF = WBF

In [0]:
def WBF(self, method_agg):
    self.eva_small = []
    self.eva_med = []


    """
    online
    """
    w_value_arr_url = "/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/w.txt"
    w_value_arr = np.loadtxt(w_value_arr_url)

    for index, group in enumerate(self.groups):
        group_rating = self.ratings[group.users, :]
        """
        Tính ma trận A
        """
        A = self.matrix_A(group, method_agg)
        

        """
        Tính sG
        """
        sG = self.cal_sG()
        
        """
        Tạo ma trận trọng số W theo công thức số [15] trong paper
        """
        """offline"""
        #rated_item = np.nonzero(np.sum(group_rating,axis = 0))
        #W_value = []
        #for item in rated_item[0]:
        #  standard_deviation = np.std(group_rating[:,item]) #tính độ lệch chuẩn của tất cả member's rating trong 
                                                                # group cho item i
        #  num_rated = (group_rating != 0).sum(0)[item] / group.size
        #  W_value.append(num_rated * (1 / (1 + standard_deviation)))
        #W = np.diag(W_value)  #chuyển thành ma trận đường chéo, có đường chéo chính là W_value
        
        """
        online 
        """
        #W_value = w_value_arr[index][~np.isnan(w_value_arr[index])][1:]
        
        W = np.diag(w_value_arr[index])
        
        """
        Tính pG - group factor, bG - bias group, Thêm vào user factor, user bias 
        """
        pG_bG = np.dot(np.linalg.inv(np.dot(np.dot(A.T, W),A) + self.lamda * np.identity(self.k + 1)),np.dot(np.dot(A.T, W), sG))
        group.group_factor = pG_bG[:-1] # lấy group factor từ #14 bỏ dòng cuối
        group.group_bias = pG_bG[-1]
        
        """
        Tính rating dự đoán của group cho tất cả item
        """
        self.predict_for_group(group, 'WBF')
        unrate = self.unrate_item(group)
            
        """
        Tạo danh sách đề xuất
        """
        self.recommend_list(group, self.pred_for_group_WBF, 'WBF',unrate)
        
        #print('virtual_user:\n',self.virtual_rating)
        #print('ma tran A:\n',A)
        #print('sG:\n',sG)
        #print('W:\n',W)
        #print('group_factor:\n',group.group_factor)
        #print('group_bias:\n', group.group_bias)
        #print('ma tran du doan group:\n', self.pred_for_group_WBF)
        #print('recommend list:\n',self.recommend_list_WBF)
        
        """
        Đánh giá
        """
        self.evaluation(group, 'WBF')
        print('Done!') 
        
    #ghi xuống file các giá trị đánh giá
    np.savetxt('eva_small_' + method_agg + '.txt', self.eva_small,fmt='%s')
    np.savetxt('eva_med_' + method_agg + '.txt', self.eva_med,fmt='%s')
    np.savetxt('eva_large_'+ method_agg + '.txt', self.eva_large,fmt='%s')
   
main.WBF = WBF

In [0]:
"""
Quá trình offline (thực hiện MF và tạo groups)
"""

b = main()

# generate group theo group size
for size in range(3, 7):
    b.generate_group(size)

#ghi danh sách group xuống file
with open('group_test1.txt', 'wb') as groups_test:
    pickle.dump(b.groups, groups_test,pickle.HIGHEST_PROTOCOL)

#load file lên:
#with open('group_test1.txt', 'rb') as groups_test:
#    config_dictionary = pickle.load(groups_test)
#    print(config_dictionary[1].size)
"""
Chạy MF để tính b_u, b_i, Q - item factor, P - user factor ban đầu
"""
# không quan trọng thứ tự chạy MF, vì  quá trình tạo virtual user của BF, WBF không bị ảnh hưởng
b.sgd()

"""
Lưu các giá trị trên vào file
"""
np.savetxt('item_factor.txt', b.Q)
np.savetxt('user_factor.txt', b.P)
np.savetxt('item_bias.txt', b.b_i)
np.savetxt('user_bias.txt', b.b_u)
np.savetxt('ratings.txt', b.ratings)
np.savetxt('test_ratings.txt', b.test_ratings)

"""
Tính ma trận dự đoán với từng user
"""
b.predicted_matrix()
np.savetxt('predicted_matrix.txt', b.predictions)

In [0]:
def prepare_run():
  b =main()
  url_Q = "/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/item_factor.txt"
  b.Q = np.loadtxt(url_Q)
  url_P = "/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/user_factor.txt"
  b.P = np.loadtxt(url_P)
  url_bi = "/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/item_bias.txt"
  b.b_i = np.loadtxt(url_bi)
  url_bu = "/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/user_bias.txt"
  b.b_u = np.loadtxt(url_bu)
  with open('/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/groups.txt', 'rb') as groups_test:
  b.groups = pickle.load(groups_test)

  url_predictions = "/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/predicted_matrix.txt"
  b.predictions = np.loadtxt(url_predictions)
  

In [0]:
"""
Chạy AF: nhớ chỉ chạy cell tính cal_virtual_user theo paper
"""

b.AF('most_pleasure')
b.AF('least_misery')
b.AF('average_weight')

In [0]:

"""
Chạy BF theo 3 cách: 3 cell cal_virtual_user, nhớ đặt lại đường dẫn lưu file để phân biệt
"""

b.BF('most_pleasure')
b.BF('least_misery')
b.BF('average_weight')

In [0]:
"""
Chạy WBF theo 3 cách: 3 cell cal_virtual_user, nhớ đặt lại đường dẫn lưu file để phân biệt
"""
b = main()

b.WBF('most_pleasure')
#b.WBF('least_misery')
#b.WBF('average_weight')

In [0]:
def BF_for_test(self, method_agg, group):
    self.eva_small = []
    self.eva_med = []
    self.eva_large = []

        
    """
    Tính ma trận A
    """
    A = self.matrix_A(group, method_agg) #trong A đã gọi hàm cal_virtual_user
        

    """
    Tính sG
    """
    sG = self.cal_sG()
        
    """
    Tính pG - group factor, bG - bias group, Thêm vào user factor, user bias 
    """
        
    pG_bG = np.dot(np.linalg.inv(np.dot(A.T,A) + self.lamda * np.identity(self.k + 1)),np.dot(A.T, sG))
    group.group_factor = pG_bG[:-1] # lấy group factor từ pG_bG bỏ dòng cuối
    group.group_bias = pG_bG[-1] # lấy dòng cuối từ pG_bG

    """
    Tính rating dự đoán của group cho tất cả item
    """
    self.predict_for_group(group, 'BF')
    unrate = self.unrate_item(group)
            
    """
    Tạo danh sách đề xuất
    """
    self.recommend_list(group, self.pred_for_group_BF, 'BF',unrate)
        
        
    """
    Đánh giá
    """
    self.evaluation(group, 'BF')
    
    np.savetxt('eva_small_' + method_agg + '.txt', self.eva_small,fmt='%s')
    np.savetxt('eva_med_' + method_agg + '.txt', self.eva_med,fmt='%s')
    np.savetxt('eva_large_'+ method_agg + '.txt', self.eva_large,fmt='%s')
main.BF_for_test = BF_for_test

In [0]:
  """
Hàm chạy thử nghiệm theo hướng ứng dụng
"""
def test():
    b=main()
    b.Q = np.loadtxt('/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/item_factor.txt')
    b.P = np.loadtxt("/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/user_factor.txt")
    b.b_i = np.loadtxt("/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/item_bias.txt")
    b.b_u = np.loadtxt("/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/user_bias.txt")
    b.predictions = np.loadtxt('/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/predicted_matrix.txt')
    ids = []
    group_size = input('Nhập số lượng người dùng trong nhóm: ')
    group_size = int(group_size)
    for user in range(int(group_size)):  
        print('Nhập id người dùng ', user +1, ": ")
        value = int(input())
        ids.append(value)
    b.groups += [group(ids, group_size)]
    #chạy phương pháp tốt nhất
    start_time = time.time()
    b.BF_for_test('most_pleasure', b.groups[-1])
    recommend_list = (np.array(b.recommend_list_BF[-1]) + 1).tolist()
    movies_url = "/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/movies.csv"
    
    movies = pd.read_csv(movies_url)
    list_recommend = movies.loc[movies['movieId'].isin(recommend_list), 'title']
    print("--- Thời gian: %s giây ---" % (time.time() - start_time))
    print(list_recommend)


In [0]:
print('Ma trận rating:\n', b.ratings)
print('Ma trận test rating:\n',b.test_ratings)
print('User factor:\n', b.P)
print('Item factor:\n', b.Q)
print('Item bias:\n',b.b_i)
print('User bias:\n', b.b_u)
print('Ma trận dự đoán:\n', b.predictions)

In [0]:
def run(self):
  w_value_arr = []
  count = 0
  for group in self.groups:
    print(count)
    count+=1
    self.cal_virtual_rating(group, "most_pleasure")
    group_rating = self.ratings[group.users, :]
    sum_rated = (group_rating != 0).sum(0)
    W_value = []
    for item in range(self.num_items):
      
      
      standard_deviation = np.std(group_rating[:,item]) #tính độ lệch chuẩn của tất cả member's rating trong 
                                                         # group cho item i
      num_rated = sum_rated[item] / group.size
      weight = num_rated * (1 / (1 + standard_deviation))
      if weight == 0:
        W_value.append(0.5)
      else:
        W_value.append(weight)
    w_value_arr.append(W_value)
  np.savetxt('test.txt', w_value_arr)  
  pd.concat([pd.DataFrame(w_value_arr)], axis=1).to_csv('w_value_arr.csv')  
main.run  = run

In [0]:
b=main()
b.Q = np.loadtxt('/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/item_factor.txt')
b.P = np.loadtxt("/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/user_factor.txt")
b.b_i = np.loadtxt("/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/item_bias.txt")
b.b_u = np.loadtxt("/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/user_bias.txt")
b.predictions = np.loadtxt('/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/predicted_matrix.txt')

In [0]:
ids = []
group_size = input('Nhập số lượng người dùng trong nhóm: ')
group_size = int(group_size)
for user in range(int(group_size)):  
  print('Nhập id người dùng ', user +1, ": ")
  value = int(input())
  ids.append(value)
b.groups += [group(ids, group_size)]
  #chạy phương pháp tốt nhất
start_time = time.time()
b.BF_for_test('most_pleasure', b.groups[-1])
recommend_list = (np.array(b.recommend_list_BF[-1]) + 1).tolist()
movies_url = "/content/drive/My Drive/LV_2015_Mai_My/Material/offline_file/movies.csv"
movies = pd.read_csv(movies_url)
movies.loc[movies['movieId'].isin([recommend_list]), 'title']
print("--- %s seconds ---" % (time.time() - start_time))
print(recommend_list)

In [0]:
test()

Nhập số lượng người dùng trong nhóm: 3
Nhập id người dùng  1 : 
366
Nhập id người dùng  2 : 
485
Nhập id người dùng  3 : 
6


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]


[3.67725664 3.51493829 3.36095217 3.73597797 3.15208724 4.46466204
 3.19679323 3.07460798 4.31853817 3.22148498]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


--- Thời gian: 1.3431921005249023 giây ---
505                                       Ghost (1990)
939                             Terminator, The (1984)
1187                                    Contact (1997)
2568                           Double Indemnity (1944)
2892    Naked Gun 2 1/2: The Smell of Fear, The (1991)
3202                              City Slickers (1991)
3348                  Man from Snowy River, The (1982)
3398                               Major League (1989)
3640                          Beautiful Mind, A (2001)
4159                        Catch Me If You Can (2002)
Name: title, dtype: object
